In [ ]:
from rdflib import URIRef, BNode, Literal, Graph, plugin, Namespace
from rdflib.serializer import Serializer 
from rdflib.namespace import FOAF, DC, RDF, RDFS, OWL, SKOS, XSD, NamespaceManager
import csv

In [ ]:
# Important:
# To Do: When using if else, which parts should still be added?
# Transitive Properties
# 


In [ ]:
kgcBase = Graph()
kgcBase.parse('./base.ttl', format='ttl')

In [ ]:
nmGraph = NamespaceManager(kgcBase)

kgc = Namespace("http://www.knowledgegraph.tech/")
nmGraph.bind("kgc", kgc)
schema = Namespace("http://schema.org/")
nmGraph.bind("schema", schema)
wd = Namespace("http://www.wikidata.org/entity/")
nmGraph.bind("wd", wd)
owl = Namespace("http://www.w3.org/2002/07/owl#")
nmGraph.bind("owl", owl)
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
nmGraph.bind("skos", skos)
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
nmGraph.bind("rdf", rdf)
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
nmGraph.bind("rdfs", rdfs)
xml = Namespace("http://www.w3.org/XML/1998/namespace")
nmGraph.bind("xml", xml) 
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
nmGraph.bind("xsd", xsd)

## Initializations

In [ ]:
# Initializations - Changes in other versions 
startFrom = 0
lastId = startFrom + 1
labelDict = {}

kgcGraph2020=kgcBase

## Conference Information

In [ ]:
#KCG = ?
KGC2020 = URIRef(("http://www.knowledgegraph.tech/conference-2020"))
kgcGraph2020.add((KGC2020, schema.sameAs, wd.Q76451254))

kgcGraph2020.add((KGC2020, schema.name, Literal("Knowledge Graph Conference 2020")))
kgcGraph2020.add((KGC2020, RDF.type, schema.EventSeries))
kgcGraph2020.add((KGC2020, RDF.type, kgc.Conference))
kgcGraph2020.add((KGC2020, schema.eventAttendanceMode, schema.OnlineEventAttendanceMode))

## Speakers

In [ ]:
with open('KGC-2020-Speakers-Recon.csv', mode='r') as csv_file:
    speaker_file = csv.DictReader(csv_file)


    for row in speaker_file:
        
        
        # Speaker's Name ---------------------------------------------------------------------------------------
        name = row['Name']
        if name:
            if name in labelDict:
                speaker = labelDict[name]
                #print("Existing Name:", name)
            else:
                newId = str(lastId).zfill(6)
                speaker = URIRef(kgc + newId)
                #print("Else Section!", name, speaker)
                lastId += 1
                #print("Incremented Last ID is ", lastId)
                kgcGraph2020.add((speaker, RDF.type, schema.Person))
                kgcGraph2020.add((speaker, schema.name, Literal(name)))
                
                if row["personQID"]:
                    #print("personQID not empty!")
                    for qid in row["personQID"].split(';'):
                        #print("Qid is ", qid)
                        kgcGraph2020.add((speaker, schema.sameAs, URIRef(wd + qid.strip())))
                labelDict[name] = speaker
         
        
        if row["LinkedIn"]:
            kgcGraph2020.add((speaker, kgc.linkedIn, Literal(row["LinkedIn"])))
            
        if row["Final Bios"]:
            kgcGraph2020.add((speaker, schema.description, Literal(row["Final Bios"])))
         
        if row["Title"]:
            kgcGraph2020.add((speaker, schema.jobTitle, Literal(row["Title"])))
         
               
        # Speaker's Country of Citizenship -------------------------------------------------------------------------- 
        countryName = row["countryOfCitizenship"]
        if countryName:
            if countryName in labelDict:
                country = labelDict[countryName]
            else:
                newId = str(lastId).zfill(6)
                country = URIRef(kgc + newId)
                #print("Else Section!", country, countryName)
                lastId += 1
                kgcGraph2020.add((country, RDF.type, schema.Country))
                kgcGraph2020.add((country, schema.name, Literal(countryName)))
            
                if row["countryOfCitizenshipQID"]:
                    #print("country of citizenship QID empty!")
                    kgcGraph2020.add((country, schema.sameAs, URIRef(wd + row["countryOfCitizenshipQID"])))
                labelDict[countryName] = country
            kgcGraph2020.add((speaker, schema.nationality, country)) 
            
            
        # Speaker's School Name  -------------------------------------------------------------------------- 
        schoolName = row["educatedAt"]
        if schoolName:
            if schoolName in labelDict:
                school = labelDict[schoolName]
            else:
                newId = str(lastId).zfill(6)
                school = URIRef(kgc + newId)
                #print("Else Section!", school, schoolName)
                lastId += 1
                kgcGraph2020.add((school, RDF.type, schema.CollegeOrUniversity))
                kgcGraph2020.add((school, schema.name, Literal(schoolName)))
                if row["educatedAtQID"]:
                    #print("education At QID not empty!")
                    kgcGraph2020.add((school, schema.sameAs, URIRef(wd + row["educatedAtQID"])))
                labelDict[schoolName] = school
            kgcGraph2020.add((speaker, schema.alumniOf, school))
    
    
    
        # Person's field of Work  -------------------------------------------------------------------------- 
        pfowName = row["personFieldOfWork"]
        if pfowName:
            #print("personFieldOfWork not empty!")
            if pfowName in labelDict:
                pfow = labelDict[pfowName]
            else:
                newId = str(lastId).zfill(6)
                pfow = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((pfow, RDF.type, kgc.FieldOfWork))
                kgcGraph2020.add((pfow, schema.name, Literal(pfowName)))
                if row["personFieldOfWorkQID"]:
                    #print("personField QID not empty!")
                    kgcGraph2020.add((pfow, schema.sameAs, URIRef(wd + row["personFieldOfWorkQID"])))
                labelDict[pfowName] = pfow
            kgcGraph2020.add((speaker, schema.knowsAbout, pfow))
            kgcGraph2020.add((speaker, kgc.hasFieldOfWork, pfow))

            
       
        # Person's Company   -------------------------------------------------------------------------- 
       
        coName = row["Company"]
        if coName:
            #print("company not empty!")
            if coName in labelDict:
                co = labelDict[coName]
            else:
                newId = str(lastId).zfill(6)
                co = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((co, RDF.type, schema.Organization))
                kgcGraph2020.add((co, schema.name, Literal(coName)))
                if row["companyQID"]:
                    #print("Company QID not empty!")
                    for qid in row["companyQID"].split(';'):
                        kgcGraph2020.add((co, schema.sameAs, URIRef(wd + qid.strip())))
                labelDict[coName] = co
            kgcGraph2020.add((speaker, schema.worksFor, co))
            
            
            
        # Consortium Name   --------------------------------------------------------------------------  
        consortiumName = row["member of"]
        if consortiumName:
            #print("member of not empty!")
            if consortiumName in labelDict:
                consortium = labelDict[consortiumName]
            else:
                newId = str(lastId).zfill(6)
                consortium = URIRef(kgc + newId)
                lastId += 1   
                kgcGraph2020.add((consortium, RDF.type, schema.Consortium))
                kgcGraph2020.add((consortium, schema.name, Literal(consortiumName)))
                if row["memberOfQID"]:
                    #print("member of QID not empty!")
                    kgcGraph2020.add((consortium, schema.sameAs, URIRef(wd + row["memberOfQID"])))
                labelDict[consortiumName] = consortium
            kgcGraph2020.add((co, schema.memberOf, consortium))
            
        
        # Additional Type Name   -------------------------------------------------------------------------------
        instanceOfBzName = row["instance of"]
        if instanceOfBzName:
             #print("instance of not empty!")
            if instanceOfBzName in labelDict:
                instanceOfBz = labelDict[instanceOfBzName]
            else:
                newId = str(lastId).zfill(6)
                instanceOfBz = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((instanceOfBz, RDF.type, kgc.BusinessType))
                kgcGraph2020.add((instanceOfBz, schema.name, Literal(instanceOfBz)))
                if row["instanceOfQID"]: 
                    #print("instance of QID not empty!")
                    kgcGraph2020.add((instanceOfBz, schema.sameAs, URIRef(wd + row["instanceOfQID"])))
                labelDict[instanceOfBzName] = instanceOfBz
            kgcGraph2020.add((co, kgc.hasBusinessType, instanceOfBz))   
            kgcGraph2020.add((co, schema.additionalType, instanceOfBz))
        
        
        # Industry Name   -------------------------------------------------------------------------------
        industryName = row["industry"]
        if industryName:
            #print("industry Name not empty!")
            if industryName in labelDict:
                industry = labelDict[industryName]
            else:
                newId = str(lastId).zfill(6)
                industry = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((industry, RDF.type, kgc.BusinessIndustry))
                kgcGraph2020.add((industry, schema.name, Literal(industryName)))
                
                if row["industryQID"]:
                    #print("industry QID not empty!")
                    kgcGraph2020.add((industry, schema.sameAs, URIRef(wd + row["industryQID"])))
                labelDict[industryName] = industry
            kgcGraph2020.add((co, kgc.hasIndustry, industry))
            kgcGraph2020.add((co, schema.additionalType, industry))
           
 

        # Company's field of Work  -------------------------------------------------------------------------- 
        cfowName = row["companyFieldOfWork"]
        if cfowName:
            #print("personFieldOfWork not empty!")
            if cfowName in labelDict:
                cfow = labelDict[cfowName]
            else:
                newId = str(lastId).zfill(6)
                cfow = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((cfow, RDF.type, kgc.FieldOfWork))
                kgcGraph2020.add((cfow, schema.name, Literal(pfowName)))
                if row["companyFieldOfWorkQID"]:
                    #print("personField QID not empty!")
                    kgcGraph2020.add((cfow, schema.sameAs, URIRef(wd + row["personFieldOfWorkQID"])))
                labelDict[cfowName] = cfow
            kgcGraph2020.add((co, schema.knowsAbout, pfow))
            kgcGraph2020.add((co, kgc.hasFieldOfWork, pfow))

      


        # Address   -------------------------------------------------------------------------------   
        
        
        AddressName = row["Address"]
        if AddressName:
            kgcGraph2020.add((co, schema.address, Literal(AddressName)))
            
              
            
            
        # City    -------------------------------------------------------------------------------   
        CityName = row["City"]
        if CityName:
            #print("HQ2 not empty!")
            if CityName in labelDict:
                City = labelDict[CityName]
            else:
                newId = str(lastId).zfill(6)
                City = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((City, schema.name, Literal(CityName)))
                kgcGraph2020.add((City, RDF.type, schema.City))
                if row["CityQID"]:
                    #print("HQ ID2 not empty!")
                    kgcGraph2020.add((City, schema.sameAs, URIRef(wd + row["CityQID"])))
                labelDict[CityName] = City
            kgcGraph2020.add((Address, schema.geoWithin, City))
            
            
        # County   -------------------------------------------------------------------------------   
        CountyName = row["County"]
        if CountyName:
            #print("HQ3 not empty!")
            if CountyName in labelDict:
                County = labelDict[CountyName]
            else:
                newId = str(lastId).zfill(6)
                County = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((County, schema.name, Literal(CountyName)))
                kgcGraph2020.add((County, RDF.type, kgc.County))
                if row["CountyQID"]:
                    #print("HQID 3 not empty!")
                    kgcGraph2020.add((County, schema.sameAs, URIRef(wd + row["CountyQID"])))
                labelDict[CountyName] = County
                kgcGraph2020.add((City, schema.geoWithin, County))
            
        
        # State   -------------------------------------------------------------------------------   
        StateName = row["State"]
        if StateName:
            #print("HQ4 not empty!")
            if StateName in labelDict:
                State = labelDict[StateName]
            else:
                newId = str(lastId).zfill(6)
                State = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((State, schema.name, Literal(StateName)))
                kgcGraph2020.add((State, RDF.type, schema.State))
                if row["StateQID"]:
                    #print("HQ ID4 not empty!")
                    kgcGraph2020.add((State, schema.sameAs, URIRef(wd + row["StateQID"])))
                labelDict[StateName] = State
                kgcGraph2020.add((County, schema.geoWithin, State))
            
        
        # Country   ------------------------------------------------------------------------------- 
        CountryName = row["Country"]
        #print("The type of Hq5 is ",type(HQ5Name))
        #if not ((HQ5Name is None) or (HQ5Name == "")):
        if CountryName:
            #print("HQ5 is not empty")
            if CountryName in labelDict:
                Country = labelDict[CountryName]
            else:
                newId = str(lastId).zfill(6)
                Country = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((Country, schema.name, Literal(CountryName)))
                kgcGraph2020.add((Country, RDF.type, schema.Country))
                if row["CountryQID"]:
                    #print("HQQID5 is not empty!")
                    kgcGraph2020.add((Country, schema.sameAs, URIRef(wd + row["CountryQID"]))) # Watch this!
                labelDict[CountryName] = Country
                kgcGraph2020.add((State, schema.geoWithin, Country)) # Decide!
            
        
        
        
        
    

## Presentations

In [ ]:
#tagsDict={}

In [ ]:
with open('KGC-2021-Presentations-Recon.csv', mode='r', encoding="utf-8") as csv_file:
    presentation_file = csv.DictReader(csv_file)

    for row in presentation_file:
        
        
        # Title + Description Presentation ------------------------------------------------------------------------------------
        
        # Think about this: a talk, and the document whichis
        title = row['Title']
        if title:
            if title in labelDict:
                talk = labelDict[title]
            else:
                newId = str(lastId).zfill(6)
                talk = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((talk, RDF.type, kgc.Talk))
                kgcGraph2020.add((talk, schema.name, Literal(title)))
                kgcGraph2020.add((talk, schema.superEvent, KGC2020))
                
                
                if row["Description"]:
                    kgcGraph2020.add((talk, schema.abstract, Literal(row["Description"])))
                labelDict[title] = talk
        
        
        # Presenter of Presentation --------------------------------------------------------------------------------
        presenterName = row['Name']
        if presenterName:
            if presenterName in labelDict:
                presenter = labelDict[presenterName]
                kgcGraph2020.add((talk, schema.performer, presenter))
                
            else:
                print("Error: Presenter {} not in the list of speakers!".format(presenterName))
                #continue
                
        
        # Tags of Presentation ------------------------------------------------------------------------------------
        tagName = row["Tag"]
        if tagName:
            if tagName in labelDict:
                tag = labelDict[tagName]
            else:
                newId = str(lastId).zfill(6)
                tag = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((tag, RDF.type, kgc.Tag))
                kgcGraph2020.add((tag, schema.name, Literal(tagName))) 
                if row["TagQID"]:
                    #print("HQQID5 is not empty!")
                    kgcGraph2020.add((tag, schema.sameAs, URIRef(wd + row["TagQID"]))) 
                   
                labelDict[tagName] = tag
            kgcGraph2020.add((talk, schema.about, tag)) 
        
        
        
        # Facet? 
        facetName = row["Facet"]
        if facetName:
            if facetName in labelDict:
                facet = labelDict[facetName]
            else:
                newId = str(lastId).zfill(6)
                facet = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((facet, RDF.type, kgc.Tag))
                kgcGraph2020.add((facet, schema.name, Literal(facetName))) 
                if row["FacetQID"]:
                    
                    kgcGraph2020.add((facet, schema.sameAs, URIRef(wd + row["FacetQID"]))) 
                   
                labelDict[facetName] = facet
            kgcGraph2020.add((tag, kgc.facetOf, facet)) 
        
        
        # Video URL of Presentation ------------------------------------------------------------------------------------
        
        videoURL = row["Video URL"]
        if videoURL:
            if videoURL in labelDict:
                video = labelDict[videoURL]
            else:
                newId = str(lastId).zfill(6)
                video = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((talk, schema.recordedIn, video))
                kgcGraph2020.add((video, RDF.type, schema.VideoObject))
                kgcGraph2020.add((video, schema.embedUrl, Literal(videoURL))) 
                
                labelDict[videoURL] = video
            kgcGraph2020.add((talk, schema.workPerformed, video))    
                
                  
        # Slides URL of Presentation ------------------------------------------------------------------------------------
        
        slidesURL = row["Slides URL"]
        if slidesURL:
            #print("Slides Not Empty!")
            if slidesURL in labelDict:
                slides = labelDict[slidesURL]
            else:
                newId = str(lastId).zfill(6)
                slides = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((slides, schema.about, talk))
                kgcGraph2020.add((slides, RDF.type, schema.PresentationDigitalDocument))
                kgcGraph2020.add((slides, schema.archivedAt, Literal(slidesURL))) 
                labelDict[slidesURL] = slides  
                
            kgcGraph2020.add((talk, schema.workPerformed, slides))

## Serialize

In [ ]:
kgc2020.serialize(destination = "kgc20-V01.ttl", format = "turtle")